# Implémenter GradCAM au CNN à la maille caractère

cf le travail de Khaled GracCam.ipynb

In [1]:
from pathlib import Path

current_dir = Path.cwd()  # this points to 'notebooks/' folder
proj_path = current_dir.parent.parent 
print(proj_path)

C:\Users\wenceslas\Documents\cours\ENSAE\2A\Normal\statapp\nlp_understanding


In [2]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from deep_nlp.cnncharclassifier import CNNCharClassifier, charToTensor
import pickle

from deep_nlp.grad_cam.utils.letter import rebuild_text, prepare_heatmap, LetterToToken
from deep_nlp.grad_cam.plot import plot_bar_heatmap, plot_text_and_heatmap

In [3]:
train_df= pd.read_csv(r"../../data/01_raw/allocine_train.csv")
test_df= pd.read_csv(r"../../data/01_raw/allocine_test.csv")
valid_df= pd.read_csv(r"../../data/01_raw/allocine_valid.csv")

model_path_saved= r"../../data/06_models/cnn_char_classifier/cnn_char_model/cnn_char_model.pt"

In [4]:
cnn_sequence_len= 1014
cnn_feature_num= 83
cnn_feature_size= 256
cnn_kernel_one= 7
cnn_kernel_two= 3
cnn_stride_one= 1
cnn_stride_two= 3
cnn_output_linear= 1024
cnn_num_class= 2
cnn_dropout= 0.5
cnn_cuda_allow= True

### Load model to CPU method

In [5]:
with open(model_path_saved, 'rb') as f:
    model_saved= pickle.load(f)

test_data= charToTensor(data_df= test_df, sentence_max_size= cnn_sequence_len)

test_load = DataLoader(test_data, batch_size= 1
                      , num_workers=4, pin_memory= True)

test_sentence= next(iter(test_load))

# Initialisation
parameters = {"sequence_len": cnn_sequence_len, "feature_num": cnn_feature_num
    , "feature_size": cnn_feature_size, "kernel_one": cnn_kernel_one
    , "kernel_two": cnn_kernel_two, "stride_one": cnn_stride_one
    , "stride_two": cnn_stride_two, "output_linear": cnn_output_linear
    , "num_class": cnn_num_class, "dropout": cnn_dropout}


model = CNNCharClassifier(**parameters)
model = torch.nn.DataParallel(model)
if cnn_cuda_allow:
    model = torch.nn.DataParallel(model).cuda()
model.load_state_dict(model_saved)
model.eval()

state_dict= model.module.module.state_dict() # delete module to allow cpu loading

cpu_model= CNNCharClassifier(**parameters).cpu()
cpu_model.load_state_dict(state_dict)

cpu_model.eval()

CNNCharClassifier(
  (before_conv): Sequential(
    (conv1_conv): Conv1d(83, 256, kernel_size=(7,), stride=(1,))
    (conv1_relu): ReLU()
  )
  (pool): Sequential(
    (conv1_maxpool): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  )
  (after_conv): Sequential(
    (conv2): Sequential(
      (0): Conv1d(256, 256, kernel_size=(7,), stride=(1,))
      (1): ReLU()
      (2): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    )
    (conv3): Sequential(
      (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
      (1): ReLU()
    )
    (conv4): Sequential(
      (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
      (1): ReLU()
    )
    (conv5): Sequential(
      (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
      (1): ReLU()
    )
    (conv6): Sequential(
      (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
      (1): ReLU()
      (2): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  

In [6]:
pred_test = []
lab = []
reviews= []
alphabet= test_data.get_alphabet()+" "

with torch.no_grad():
    for review, label in test_load:
        pred_test.append(torch.exp(cpu_model(review)))
        lab.append(label.float())
        reviews.append(rebuild_text(text= review
                                 , alphabet= alphabet
                                 , space_index= 83
                                 , sequence_len= cnn_sequence_len))
        
pred_test = torch.cat(pred_test)
lab = torch.cat(lab)

In [7]:
results= pd.DataFrame({
    "review": reviews
    , "label": lab
    , "prediction_prob_1":  pred_test[:,1]
    , "prediction_label": [1.0 if i > 0.5 else 0.0 for i in pred_test[:,1]]
    , "true_review": test_df["review"].values
#     , "heatmap_test_normalized_0": heatmap_test_normalized_0
#     , "heatmap_test_normalized_1": heatmap_test_normalized_1.tolist()
#     , "heatmap_test_max_0": heatmap_test_max_0.tolist()
#     , "heatmap_test_max_1": heatmap_test_max_1.tolist()
})

# del heatmap_test_normalized_1, heatmap_test_max_0, heatmap_test_max_1

print(results.shape)
results.head()

(20000, 5)


,review,label,prediction_prob_1,prediction_label,true_review
0,"Magnifique épopée, une belle histoire, touchan...",1.0,0.997272,1.0,"Magnifique épopée, une belle histoire, touchan..."
1,Je n'ai pas aimé mais pourtant je lui mets 2 é...,0.0,0.018032,0.0,Je n'ai pas aimé mais pourtant je lui mets 2 é...
2,Un dessin animé qui brille par sa féerie et se...,1.0,0.804264,1.0,Un dessin animé qui brille par sa féerie et se...
3,"Si c'est là le renouveau du cinéma fran ais, c...",0.0,0.004019,0.0,"Si c'est là le renouveau du cinéma français, c..."
4,Et pourtant on s en Doutait !Second volet très...,0.0,0.000941,0.0,Et pourtant on s’en Doutait !Second volet très...


In [8]:
#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds[:,1])    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    
    return acc

acc= binary_accuracy(pred_test, lab.float())
print("ACC on test set : {}".format(acc))

ACC on test set : 0.9117000102996826


### Study model errors

In [9]:
# When the model is wrong what is the MAE & RMSE
mae= np.mean(np.abs(results["label"] - results["prediction_prob_1"]))
print("MAE : {}".format(mae))
rmse= np.sqrt(np.mean((results["label"] - results["prediction_prob_1"])**2))
print("RMSE : {}".format(rmse))

MAE : 0.1163390651345253
RMSE : 0.2559721767902374


In [10]:
threshold= 0.85
result_worse_pred= results[np.abs(results["label"] - results["prediction_prob_1"]) >= threshold]
print(result_worse_pred.shape)
print(result_worse_pred.describe())
result_worse_pred.head()

(670, 5)
            label  prediction_prob_1  prediction_label
count  670.000000         670.000000        670.000000
mean     0.601493           0.412906          0.398507
std      0.489957           0.423401          0.489957
min      0.000000           0.000069          0.000000
25%      0.000000           0.054643          0.000000
50%      1.000000           0.117460          0.000000
75%      1.000000           0.910351          1.000000
max      1.000000           0.999778          1.000000


,review,label,prediction_prob_1,prediction_label,true_review
16,Eli Roth est un réalisateur intéressant et red...,1.0,0.135620,0.0,Eli Roth est un réalisateur intéressant et red...
23,"Il y a maintenant presque 50ans, un petit ciné...",0.0,0.962284,1.0,"Il y a maintenant presque 50ans, un petit ciné..."
46,Cécile de France crève l écran en Marquise de ...,1.0,0.083666,0.0,Cécile de France crève l’écran en Marquise de ...
49,"C est dr le, follement dr le car les personnag...",1.0,0.112146,0.0,"C est drôle, follement drôle car les personnag..."
50,"Après la Une de ""Minute"", je ne peux que recom...",1.0,0.056054,0.0,"Après la Une de ""Minute"", je ne peux que recom..."


In [11]:
# rebuild dataloader
worst_data= charToTensor(data_df= result_worse_pred[["review", "label"]].copy(), sentence_max_size= cnn_sequence_len)

worst_load = DataLoader(worst_data, batch_size= 1
                      , num_workers=4, pin_memory= True)

In [23]:
pred_test = []
lab = []
reviews= []
alphabet= test_data.get_alphabet()+" "
heatmap= []

for review, label in worst_load:
    
    label_inv= 0 if label == 1 else 1
    
    pred_test.append(torch.exp(cpu_model(review)))
    lab.append(label.float())
    reviews.append(rebuild_text(text= review
                             , alphabet= alphabet
                             , space_index= 83
                             , sequence_len= cnn_sequence_len))
    heatmap.append(cpu_model.get_heatmap(text= review
                                    , num_class= label_inv
                                    , dim= [0, 2]
                                    , type= "normalized"))
        
pred_test = torch.cat(pred_test)
lab = torch.cat(lab)

tensor([[[ 0.0125,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0181,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0054,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.0364,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000, -0.0039,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0383,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])
torch.Size([1, 256, 1008])
tensor([[[ 0.0000e+00,  0.0000e+00, -3.7210e-03,  ...,  0.0000e+00,
          -5.1706e-05,  0.0000e+00],
         [ 4.4227e-03,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           1.0625e-04,  0.0000e+00],
         [-1.5631e-03,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          -1.3263e-04,  0.0000e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00,  3.7312e-03,  ...,  7.1406e-05,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  4.8222e-04,  0.0000e+00,  ...,  0.0000e+00,
          -1.1420e-04,  0.000

tensor([[[ 0.0123,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.0252,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0111,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000, -0.0299,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.0008,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000, -0.0322,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])
torch.Size([1, 256, 1008])
tensor([[[ 2.5136e-04,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [-9.3938e-05,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 9.5664e-05,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [-4.4626e-04,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 4.4336e-05,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.000

tensor([[[-0.0092,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0235,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000, -0.0080,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0295,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0003,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0269,  ...,  0.0000,  0.0000,  0.0000]]])
torch.Size([1, 256, 1008])
tensor([[[ 0.0012,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0026,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0007,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.0030,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000, -0.0005,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0033,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])
torch.Size([1, 256, 1008])
tensor([[[ 0.0000,  0.0000, -0.0015,  ...,  0.0000,  0.0000,  

tensor([[[ 0.0000,  0.0276,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0243,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0125,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000, -0.0490,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000, -0.0111,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000, -0.0558,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])
torch.Size([1, 256, 1008])
tensor([[[ 0.0415,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0600,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0576,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000, -0.0625,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0175,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000, -0.0613,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])
torch.Size([1, 256, 1008])
tensor([[[ 0.0000,  0.0021,  0.0000,  ...,  0.0000,  0.0000,  

tensor([[[-0.0151,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0350,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0170,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0334,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0050,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0458,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])
torch.Size([1, 256, 1008])
tensor([[[ 0.0122,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0095,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0019,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.0209,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0115,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0145,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])
torch.Size([1, 256, 1008])
tensor([[[ 0.0000,  0.0000, -0.0691,  ...,  0.0000,  0.0000,  

...

tensor([[[ 0.0000e+00,  0.0000e+00, -1.4046e-04,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 2.4152e-04,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -8.3106e-05,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 2.7335e-04,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 2.3027e-05,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 2.8414e-04,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]]])
torch.Size([1, 256, 1008])
tensor([[[ 3.3884e-03,  0.0000e+00,  0.0000e+00,  ..., -5.6826e-05,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -3.9093e-03,  ...,  0.0000e+00,
           0.0000e+00, -3.8207e-05],
         [ 1.4374e-03,  0.0000e+00,  0.0000e+00,  ...,  1.3148e-04,
           0.0000e+00,  0.0000e+00],
         ...

tensor([[[ 0.0107,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0118,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0050,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.0144,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0024,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0066,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])
torch.Size([1, 256, 1008])
tensor([[[-4.3155e-03,  0.0000e+00,  0.0000e+00,  ..., -6.6527e-06,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  3.3886e-03,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  2.4933e-05],
         [ 0.0000e+00,  0.0000e+00, -1.5418e-03,  ..., -3.9245e-05,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 5.5594e-03,  0.0000e+00,  0.0000e+00,  ...,  1.9270e-05,
           0.0000e+00,  0.0000e+00],
         [-5.0635e-04,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00, -3.032

tensor([[[ 1.8777e-03,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -1.3375e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00, -7.4754e-05,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00, -8.9383e-03,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  2.4246e-04,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [-1.6864e-02,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]]])
torch.Size([1, 256, 1008])
tensor([[[-0.0220,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0456,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0024,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0510,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  

tensor([[[-3.5122e-03,  0.0000e+00,  0.0000e+00,  ..., -2.3023e-06,
           0.0000e+00,  0.0000e+00],
         [ 4.6031e-03,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00, -1.5112e-06],
         [ 0.0000e+00,  0.0000e+00, -3.8883e-04,  ...,  0.0000e+00,
           0.0000e+00, -4.0207e-05],
         ...,
         [ 6.9950e-03,  0.0000e+00,  0.0000e+00,  ...,  3.7958e-05,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00, -3.8900e-03,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00, -3.8108e-05],
         [ 7.6137e-03,  0.0000e+00,  0.0000e+00,  ...,  2.6456e-05,
           0.0000e+00,  0.0000e+00]]])
torch.Size([1, 256, 1008])
tensor([[[ 0.0000,  0.0035,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0034,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0008,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.0102,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  

tensor([[[-0.0060,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0171,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0029,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0148,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000, -0.0020,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0189,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])
torch.Size([1, 256, 1008])
tensor([[[-0.0005,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0011,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000, -0.0002,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0013,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0004,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0015,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])
torch.Size([1, 256, 1008])
tensor([[[ 0.0000,  0.0000, -0.0010,  ...,  0.0000,  0.0000,  

In [24]:
review_rebuild= result_worse_pred["review"].values
tokens= []
cleaned_tokens= []
heatmap_test= []
i= 0
for r, h in zip(review_rebuild, heatmap):
    prepared_heatmap= prepare_heatmap(heatmap= h, text= r)
    letter_to_token= LetterToToken(text= r
                               , heatmap= prepared_heatmap)
    try:
        results_dict= letter_to_token.transform_letter_to_token(type= "tanh")
        
        tokens.append(results_dict["tokens"])
        cleaned_tokens.append(results_dict["cleaned_tokens"])
        heatmap_test.append(results_dict["heatmap"])
    except:
        print(r)
        print(i)
        print(prepared_heatmap)
    i+=1
#     tokens= results_dict["tokens"]
#     heatmap_test= results_dict["heatmap"]

In [28]:
%matplotlib
indexed_test= 0
seuil_heatmap= 0.75
h= heatmap_test[indexed_test]
plot_text_and_heatmap(text= tokens[indexed_test]
                      , heatmap= np.where(np.abs(h) >= seuil_heatmap, h, 0)
                      , figsize=(7, 7)
                      , cmap= "PiYG"
                      , word_or_letter= "word")
pred_test[indexed_test]

Using matplotlib backend: TkAgg


tensor([0.8644, 0.1356], grad_fn=<SelectBackward>)

In [29]:
plot_text_and_heatmap(text= tokens[indexed_test]
                      , heatmap= heatmap_test[indexed_test]
                      , figsize=(7, 7)
                      , cmap= "PiYG"
                      , word_or_letter= "word")
pred_test[indexed_test]

tensor([0.8644, 0.1356], grad_fn=<SelectBackward>)

### Compute GradCam on special prediction case